In [ ]:
import numpy as np
from __future__ import division

In [ ]:
import picos as pic
import cvxopt as cvx

In [ ]:
from scipy.special import comb as nchoosek # n Choose k (n ueber k)

# As seen in A.4 jasniwiecz
# Intervalltransformation
def transQ(Q_in, a, b):
    m = len(Q_in)
    n = Q_in[0].size[0]
    for i1 in range(0, m-1):
        Q_out[i1] = Q_in[i1]
        for i2 in range(i1+1, m-1):
            Q_out[i1] = Q_out[i1] + b**(i2-i1) * nchoosek(i2, i1) * Q_in[i2]
        Q_out[i1] = a**i1*Q_out[i1]
    return Q_out

def calcQSum(Q_in):
    m = len(Q_in)
    n = Q_in[0].size[0]
    if m is 1:
        Q_sum = [[2*Q_in[0], np.zeros(n)], [np.zeros(n), np.zeros(n)]]
    elif m is 2:
        Q_sum = [[2*Q_in[0], Q_in[1]], [Q_in[1], np.zeros(n)]]
    else:
        Q_sum = [[2*Q_in[0], Q_in[1]], [Q_in[1], 2*Q_in[2]]]

    for i1 in range(4, m+1, 2):
        if i1 is not m:
            Q_sum = [[Q_sum], [[np.zeros((i1/2-1)*n, n)], [Q_in[i1]]], 
                     [np.zeros(n, (i1/2-1)), Q_in[i1], 2*Q_in[i1+1]]]
        else:
            Q_sum = [[Q_sum], [[np.zeros((i1/2-1)*n, n)], [Q_in[i1]]], 
                     [np.zeros(n, (i1/2-1)), Q_in[i1], np.zeros(n)]]
    Q_sum = 0.5*Q_sum
    return Q_sum

In [ ]:
################
# Demo Picos   #
################

#generate data
A = [   cvx.sparse([[0 ,2 ,-1],
                    [-1,0 ,2 ],
                    [0 ,1 ,0 ]]),
        cvx.sparse([[1 ,2 ,0 ],
                    [2 ,0 ,0 ]]),
        cvx.sparse([[0 ,2 ,2 ]]),
    ]
K = cvx.sparse([[1 ,1 ,1 ],
                [1 ,-5,-5]])

#size of the data
s = len(A)
m = A[0].size[0]
l = [ Ai.size[1] for Ai in A ]
r = K.size[1]


#creates a problem and the optimization variables
prob = pic.Problem()
mu = prob.add_variable('mu',s)
Z  = [prob.add_variable('Z[' + str(i) + ']', (l[i],r))
      for i in range(s)]

#convert the constants into params of the problem
A = pic.new_param('A',A)
K = pic.new_param('K',K)

#add the constraints
prob.add_constraint( pic.sum([ A[i]*Z[i] for i in range(s)], #summands
                            'i',                            #name of the index
                            '[s]'                           #set to which the index belongs
                           ) == K
                   )
prob.add_list_of_constraints( [ abs(Z[i]) < mu[i] for i in range(s)], #constraints
                              'i',                                    #index of the constraints
                              '[s]'                                   #set to which the index belongs
                            )

#sets the objective
prob.set_objective('min', 1 | mu ) # scalar product of the vector of all ones with mu

#display the problem
print prob

#call to the solver cvxopt
sol = prob.solve(solver='cvxopt', verbose = 0)

#show the value of the optimal variable
print '\n  mu ='
print mu

#show the dual variable of the equality constraint
print'\nThe optimal dual variable of the'
print prob.get_constraint(0)
print 'is :'
print prob.get_constraint(0).dual

In [ ]:
################
# Demo 2 Picos #
################

import numpy as s
def testAx_less_b():
    A = cvx.matrix(s.eye(3))
    c = cvx.matrix(s.ones( (3,1) ) )

    prob=pic.Problem()
    AA = pic.new_param('A', A)
    cc = pic.new_param('c', c)
    bb = pic.new_param('b', s.ones( (3,1) ))
    xx = prob.add_variable('x', 3)

    prob.set_objective('max', cc.T*xx)
    prob.add_constraint(AA*xx < bb)
    print prob

    #    prob.solve(verbose=1, solver='cvxopt')
    prob.solve()
    x = xx.value
    print x
testAx_less_b()